# Problema do despacho térmico

Existem $n$ usinas termelétricas e cada usina $i$ tem capacidade máxima $G_i$ e custo $c_i$. Dada uma demanda $d$, qual a melhor maneira de escolher as capacidades de operação $g_i$ de cada usina de modo a minimizar o custo $C$?

A ideia aqui é esgotar as capacidades de operação $g$ das usinas mais baratas, até atingir a demanda $d$.

In [ ]:
import Pkg; Pkg.add("DataFrames")

In [137]:
using DataFrames

function despacho_termico(G_i, c_i, d)
    is = collect(range(1, stop = length(G_i)))
    Gc = collect(zip(is, G_i, c_i))
    sort!(Gc, by = x -> x[3])
    used = []
    while d > 0
        if d >= Gc[1][2]
            d -= Gc[1][2]
            bye = popfirst!(Gc)
            push!(used, bye)
        else
            push!(used, (Gc[1][1], d, Gc[1][3]))
            d = 0
        end
    end
    sort!(used, by = x -> x[1])
    usines = DataFrame(usines = (used -> used[1]).(used), capacity_used = (used -> used[2]).(used), cost = (used -> used[3]).(used))
    return usines
end;

In [138]:
G_i = [100, 200, 50, 10, 109, 1] #Usinas i na ordem
c_i = [2, 100, 1, 1, 1, 99]
d = 320

used = despacho_termico(G_i, c_i, d)

,usines,capacity_used,cost
,Int64,Int64,Int64
1,1,100,2
2,2,50,100
3,3,50,1
4,4,10,1
5,5,109,1
6,6,1,99


# Problema da maximização da capacidade disponível mínima do ano (agenda de manutenção)

Dado um conjunto de capacidades máximas $G_i$ de usinas $i$, a soma das capacidades das usinas que estão operando em um determinado mês é a capacidade disponível desse mês. Cada usina deve entrar em manutenção uma vez por ano. Deve-se distribuir as as paradas de manutenção das usinas durante um ano em uma agenda de manutenção, de modo que a $\textbf{menor das capacidades disponíveis dos meses seja maximizada}$.

A ideia neste caso é distribuir as doze usinas com maiores capacidades nos doze meses. A partir disso, distribuir as usinas restantes, cada uma, no mês que atualmente tem maior capacidade disponível.

In [81]:
#=function usines_in_a_year2(G)
    G = copy(G)
    capacity = sum(G)
    sort!(G, rev = true)
    months = [[month, capacity - G[month], G[month]] for month in 1:12]
    G = copy(G[13:end])
    for usine in G
        sort!(months, by = month -> month[2], rev = true)
        push!(months[1], usine)
        months[1][2] -= usine
    end
    sort!(months, by = month -> month[1])
    months = DataFrame(month = (month -> month[1]).(months), available_capacity = (month -> month[2]).(months), capacities_used = (month -> month[3:end]).(months))
    return months
end=#

In [87]:
import Pkg; Pkg.add("RandomNumbers")

 Resolving package versions...
 Installed RandomNumbers ─ v1.0.3
  Updating `C:\Users\lucasresck\.julia\environments\v1.0\Project.toml`
  [e6cf234a] + RandomNumbers v1.0.3
  Updating `C:\Users\lucasresck\.julia\environments\v1.0\Manifest.toml`
  [e6cf234a] + RandomNumbers v1.0.3
  Building RandomNumbers → `C:\Users\lucasresck\.julia\packages\RandomNumbers\cr9ta\deps\build.log`


In [133]:
using Random

function usines_in_a_year2(G)
    capacity = sum(G)
    is = collect(range(1, stop = length(G)))
    G = collect(zip(is, G))
    sort!(G, by = G -> G[2], rev = true)
    months = [[month, capacity - G[usine][2], [G[usine][2]], [G[usine][1]]] for (month, usine) in zip(collect(1:12), shuffle(collect(1:12)))]
    G = copy(G[13:end])
    for usine in G
        sort!(months, by = month -> month[2], rev = true)
        push!(months[1][3], usine[2])
        push!(months[1][4], usine[1])
        months[1][2] -= usine[2]
    end
    sort!(months, by = month -> month[1])
    months = DataFrame(month = (month -> month[1]).(months), available_capacity = (month -> month[2]).(months), capacities_used = (month -> month[3]).(months), usines_used = (month -> month[4]).(months))
    return months
end;

In [139]:
G = [10, 5, 20, 18, 23, 32, 7, 12, 10, 20, 17, 32, 12, 13, 7]
months = usines_in_a_year2(G)

,month,available_capacity,capacities_used,usines_used
,Int64,Int64,Array…,Array…
1,1,225,[13],[14]
2,2,206,[32],[6]
3,3,221,"[12, 5]","[13, 2]"
4,4,226,[12],[8]
5,5,215,[23],[5]
6,6,220,[18],[4]
7,7,221,"[10, 7]","[9, 7]"
8,8,221,"[10, 7]","[1, 15]"
9,9,218,[20],[10]


In [140]:
G2 = [23, 23, 6, 23, 23, 12, 23, 23, 4, 23, 23, 23, 17, 11, 19]
months = usines_in_a_year2(G2)

,month,available_capacity,capacities_used,usines_used
,Int64,Int64,Array…,Array…
1,1,253,[23],[7]
2,2,253,[23],[11]
3,3,253,[23],[2]
4,4,253,[23],[1]
5,5,253,"[19, 4]","[15, 9]"
6,6,253,[23],[5]
7,7,253,"[17, 6]","[13, 3]"
8,8,253,[23],[8]
9,9,253,[23],[12]


In [141]:
G3 = rand(20:32, 15)
@show G3
months = usines_in_a_year2(G3)

G3 = [22, 30, 28, 24, 23, 32, 24, 32, 29, 26, 24, 29, 20, 24, 24]


,month,available_capacity,capacities_used,usines_used
,Int64,Int64,Array…,Array…
1,1,359,[32],[8]
2,2,363,[28],[3]
3,3,361,[30],[2]
4,4,344,"[24, 23]","[4, 5]"
5,5,345,"[24, 22]","[11, 1]"
6,6,362,[29],[9]
7,7,365,[26],[10]
8,8,362,[29],[12]
9,9,347,"[24, 20]","[14, 13]"
